In [1]:
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
import re
from darts import datasets
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
from serialize import deserialize_str,SerializerSettings
import matplotlib.pyplot as plt
from statistics import mean
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
from datasets import load_dataset
settings =SerializerSettings(base=10, prec=3, signed=True, time_sep=', ', bit_sep=' ')
from sklearn.preprocessing import StandardScaler

C:\Users\Petro\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
settings =SerializerSettings(base=10, prec=3, signed=True, time_sep=', ', bit_sep=' ')

def min_max_scale_inv(array,values_inver, min, max):
    scaler = MinMaxScaler()
     
    if isinstance(array, int) :
        X_train_help = np.append(array, min)
        X_train_help= np.append(X_train_help, max)
        scaler.fit(X_train_help.reshape(-1, 1))
        x_scaled = scaler.transform(X_train_help.reshape(-1, 1))[0]
        in_values = scaler.inverse_transform(values_inver.reshape(-1, 1))

    else :
        
        # give the min max to the array, if they already have it don't change anything, 
        #if they do not have it it will change the scaling to the min max of the whole domain
        X_train_help = np.append(array, min)
        X_train_help= np.append(X_train_help, max)
        scaler.fit(X_train_help.reshape(-1, 1))
        x_scaled = scaler.transform(array.reshape(-1, 1)) 
        in_values= scaler.inverse_transform(values_inver.reshape(-1, 1))
        #x_scaled = x_scaled
    return in_values

def min_max_scale(array, min, max):
    scaler = MinMaxScaler()
     
    if isinstance(array, int) :
        X_train_help = np.append(array, min)
        X_train_help= np.append(X_train_help, max)
        scaler.fit(X_train_help.reshape(-1, 1))
        x_scaled = scaler.transform(X_train_help.reshape(-1, 1))[0]

    else :
        
        # give the min max to the array, if they already have it don't change anything, 
        #if they do not have it it will change the scaling to the min max of the whole domain
        X_train_help = np.append(array, min)
        X_train_help= np.append(X_train_help, max)
        scaler.fit(X_train_help.reshape(-1, 1))
        x_scaled = scaler.transform(array.reshape(-1, 1))  
        x_scaled = x_scaled
    return x_scaled

# Load in Datasets

## Load Prompt1 

In [5]:
prompt1 = pd.read_pickle("prompt1/test_results.pkl")
path = "norm_enc/Normalized_Encoded_final_test_dataset.json"

enc_dataset = load_dataset("json", data_files=path, split="train")

mse_in_values = pd.read_pickle("test_mse_calc.pkl")

In [6]:
prompt1

,dataset_name,answer
0,ETTh1Dataset0,"[0.0005, 0.6465, 0.6154999999999999, 0.6345, 0..."
1,ETTh1Dataset1,"[0.0005, 0.6094999999999999, 0.565499999999999..."
2,ETTh1Dataset2,"[0.0005, 0.6094999999999999, 0.609499999999999..."
3,ETTh1Dataset3,"[0.0005, 0.7424999999999999, 0.771499999999999..."
4,ETTh1Dataset4,"[0.0005, 0.6074999999999999, 0.575499999999999..."
5,ETTh1Dataset5,"[0.0005, 0.6084999999999999, 0.5765, 0.5465, 0..."
6,ETTh2Dataset0,"[0.0005, 0.6635, 0.6635, 0.6775, 0.6615, 0.643..."
7,ETTh2Dataset1,"[0.6405, 0.6405, 0.6405, 0.6405, 0.6405, 0.640..."
8,ETTh2Dataset2,"[0.0005, 0.14850000000000002, 0.1125, 0.6295, ..."
9,ETTh2Dataset3,"[0.0005, 0.6345, 0.6694999999999999, 0.6365, 0..."


In [7]:
mse_prompt =  pd.DataFrame(columns=["dataset_name","mse","mae","mape"])

In [8]:

for i in range(len(prompt1)):
      
    inv_values = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],prompt1["answer"][i],mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
    
    
    #x=deserialize_str(dataset["season_forcasting"][i],settings) + deserialize_str(dataset['remain_forcasting'][i],settings)+ deserialize_str(dataset['trend_forcasting'][i],settings)
    x= deserialize_str(enc_dataset["output"][i],settings)
    
    inv_values_x = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],x,mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
    
    mae_value_norm     = mean_absolute_error(inv_values_x,inv_values)
    mape_norm  = mean_absolute_percentage_error(inv_values_x,inv_values)
    value_mse_norm  = mean_squared_error(inv_values_x,inv_values)

    
    new_row = {"dataset_name": enc_dataset["dataset_name"][i] ,"mse": value_mse_norm , "mae":mae_value_norm ,"mape":mape_norm  }
    mse_prompt.loc[len(mse_prompt)] = new_row
    mse_prompt = mse_prompt.reset_index(drop=True)

In [9]:
mse_prompt

,dataset_name,mse,mae,mape
0,ETTh1Dataset0,118.417213,9.407470,3.761953
1,ETTh1Dataset1,7.643811,1.984469,0.407866
2,ETTh1Dataset2,81.785793,7.543683,1.063378
3,ETTh1Dataset3,13.305599,3.263074,1.631910
4,ETTh1Dataset4,10.425015,3.019721,0.955083
5,ETTh1Dataset5,2.602684,1.493749,1.060210
6,ETTh2Dataset0,904.640053,26.005891,0.825496
7,ETTh2Dataset1,42.814517,6.311752,0.662190
8,ETTh2Dataset2,5060.897592,59.449485,1.399251
9,ETTh2Dataset3,18.404117,3.241929,0.310460


In [23]:

mse_prompt['dataset_name'] = mse_prompt['dataset_name'].str.slice(0, -1)
grouped_prompt1 = mse_prompt.groupby('dataset_name').mean().reset_index()
grouped_prompt1


,dataset_name,mse,mae,mape
0,ETTh1Datas,39.030019,4.452028,1.480067
1,ETTh2Datas,1093.853861,20.971104,68.338183
2,ETTm1Datas,187.752976,8.553157,3.208797
3,ETTm2Datas,266.910030,13.254457,21.434919
4,HeartRateDatas,67.606655,5.767958,0.056295
5,SunspotsDatas,7283.105285,75.194814,1.052961


## Load Prompt2

In [10]:
prompt2 = pd.read_pickle("prompt2/test_results.pkl")
path = "norm_enc/Normalized_Encoded_final_test_dataset.json"

enc_dataset = load_dataset("json", data_files=path, split="train")

mse_in_values = pd.read_pickle("test_mse_calc.pkl")

In [11]:
mse_prompt2 =  pd.DataFrame(columns=["dataset_name","mse","mae","mape"])

In [12]:

for i in range(len(prompt1)):
      
    inv_values = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],prompt2["answer"][i],mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
    
    
    #x=deserialize_str(dataset["season_forcasting"][i],settings) + deserialize_str(dataset['remain_forcasting'][i],settings)+ deserialize_str(dataset['trend_forcasting'][i],settings)
    x= deserialize_str(enc_dataset["output"][i],settings)
    
    inv_values_x = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],x,mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
    
    mae_value_norm     = mean_absolute_error(inv_values_x,inv_values)
    mape_norm  = mean_absolute_percentage_error(inv_values_x,inv_values)
    value_mse_norm  = mean_squared_error(inv_values_x,inv_values)

    
    new_row = {"dataset_name": enc_dataset["dataset_name"][i] ,"mse": value_mse_norm , "mae":mae_value_norm ,"mape":mape_norm  }
    mse_prompt2.loc[len(mse_prompt2)] = new_row
    mse_prompt2 = mse_prompt2.reset_index(drop=True)

In [13]:
mse_prompt2

,dataset_name,mse,mae,mape
0,ETTh1Dataset0,78.773512,6.206686,3.217867
1,ETTh1Dataset1,3.705166,1.397442,0.395957
2,ETTh1Dataset2,82.330850,6.437636,1.588468
3,ETTh1Dataset3,4.422388,1.489363,0.806435
4,ETTh1Dataset4,4.325780,1.772208,0.554012
5,ETTh1Dataset5,696.992415,3.618477,3.279740
6,ETTh2Dataset0,1844.053589,41.884552,1.298854
7,ETTh2Dataset1,96.305311,8.526533,0.883011
8,ETTh2Dataset2,1079.943272,32.461395,0.729078
9,ETTh2Dataset3,60.534770,6.394329,0.633090


In [14]:

mse_prompt2['dataset_name'] = mse_prompt2['dataset_name'].str.slice(0, -1)
grouped_prompt2 = mse_prompt2.groupby('dataset_name').mean().reset_index()
grouped_prompt2


,dataset_name,mse,mae,mape
0,ETTh1Dataset,145.091685,3.486969,1.640413
1,ETTh2Dataset,561.878266,18.281526,18.966332
2,ETTm1Dataset,14138.786792,14.755573,4.766884
3,ETTm2Dataset,22422.731166,27.133697,21.412699
4,HeartRateDataset,3196.481991,18.510778,0.182025
5,SunspotsDataset,11273.702008,98.728200,1.212876


## Load prompt 3

In [15]:
norm_df = pd.read_pickle("norm_enc/test_results.pkl")
path = "norm_enc/Normalized_Encoded_final_test_dataset.json"

norm_dataset = load_dataset("json", data_files=path, split="train")
mse_in_values = pd.read_pickle("test_mse_calc.pkl")


In [16]:
mse_norm_table =  pd.DataFrame(columns=["dataset_name","mse","mae","mape"])

In [17]:

for i in range(len(norm_df)):
      
    inv_values = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],norm_df["answer"][i],mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
    
    
    #x=deserialize_str(dataset["season_forcasting"][i],settings) + deserialize_str(dataset['remain_forcasting'][i],settings)+ deserialize_str(dataset['trend_forcasting'][i],settings)
    x= deserialize_str(norm_dataset["output"][i],settings)
    
    inv_values_x = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],x,mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
    
    mae_value_norm     = mean_absolute_error(inv_values_x,inv_values)
    mape_norm  = mean_absolute_percentage_error(inv_values_x,inv_values)
    value_mse_norm  = mean_squared_error(inv_values_x,inv_values)

    
    new_row = {"dataset_name": norm_dataset["dataset_name"][i] ,"mse": value_mse_norm , "mae":mae_value_norm ,"mape":mape_norm  }
    mse_norm_table.loc[len(mse_norm_table)] = new_row
    mse_norm_table = mse_norm_table.reset_index(drop=True)

In [19]:
mse_norm_table

,dataset_name,mse,mae,mape
0,ETTh1Dataset,672.999233,24.454891,8.791265
1,ETTh1Dataset,7.063060,2.260409,0.619964
2,ETTh1Dataset,39.921289,4.726494,1.417496
3,ETTh1Dataset,3.477295,1.688049,0.856949
4,ETTh1Dataset,4.749643,1.926633,0.599890
5,ETTh1Dataset,0.964221,0.808512,0.606714
6,ETTh2Dataset,460.078752,20.801280,0.610181
7,ETTh2Dataset,179.272620,12.451257,1.262785
8,ETTh2Dataset,432.813780,18.691448,0.434975
9,ETTh2Dataset,297.538561,17.182424,1.590883


In [18]:

mse_norm_table['dataset_name'] = mse_norm_table['dataset_name'].str.slice(0, -1)
grouped_norm = mse_norm_table.groupby('dataset_name').mean().reset_index()
grouped_norm


,dataset_name,mse,mae,mape
0,ETTh1Dataset,121.529124,5.977498,2.148713
1,ETTh2Dataset,279.262708,14.520359,11.850624
2,ETTm1Dataset,66.574093,4.823614,2.749328
3,ETTm2Dataset,1218.696575,26.005910,17.967922
4,HeartRateDataset,41.164577,5.475488,0.053061
5,SunspotsDataset,7025.678528,73.864552,1.087073


In [58]:
bar_colors = ['tab:red', 'tab:blue', 'tab:orange','tab:green']
#bar_labels = [str(grouped["mse"][0]), str(grouped["mse"][1]), str(grouped["mse"][2]), str(grouped["mse"][3]),str(grouped["mse"][4]),str(grouped["mse"][5])]

fig, ax = plt.subplots(2,3) 
fig.set_figheight(15)
fig.set_figwidth(15)



#ETTh1
etth1= [grouped_prompt1["mse"][0],grouped_prompt2["mse"][0],grouped_norm["mse"][0]]
etth1l = [np.round(grouped_prompt1["mse"][0],2),np.round(grouped_prompt2["mse"][0],2),np.round(grouped_norm["mse"][0],2)]
etth1n= ["prompt1","prompt2","prompt3"]
ax[0, 0].bar(etth1n, etth1,color  =bar_colors, edgecolor="k" ,label= etth1l)
ax[0, 0].grid(False)
ax[0, 0].set_ylabel('MSE Values')
ax[0, 0].set_xlabel('Data Processing')

ax[0, 0].set_title('MSE from ETTh1')
ax[0, 0].legend(title='MSE Values')

ax[0, 0].spines['top'].set_visible(True)
ax[0, 0].spines['right'].set_visible(True)
ax[0, 0].spines['bottom'].set_visible(True)
ax[0, 0].spines['left'].set_visible(True)





#ETTh2
etth2= [grouped_prompt1["mse"][1],grouped_prompt2["mse"][1],grouped_norm["mse"][1]]
etth2l = [np.round(grouped_prompt1["mse"][1],2),np.round(grouped_prompt2["mse"][1],2),np.round(grouped_norm["mse"][1],2)]
etth2n= ["prompt1","prompt2","prompt3"]
ax[0, 1].bar(etth2n, etth2,color  =bar_colors,edgecolor="k",label= etth2l)
ax[0, 1].grid(False)
ax[0, 1].set_ylabel('MSE Values')
ax[0, 1].set_xlabel('Data Processing')

ax[0, 1].set_title('MSE from ETTh2')
ax[0, 1].legend(title='MSE Values')

ax[0, 1].spines['top'].set_visible(True)
ax[0, 1].spines['right'].set_visible(True)
ax[0, 1].spines['bottom'].set_visible(True)
ax[0, 1].spines['left'].set_visible(True)




#ETTm1
ettm1= [grouped_prompt1["mse"][2],grouped_prompt2["mse"][2],grouped_norm["mse"][2]]
ettm1n= ["prompt1","prompt2","prompt3"]
ettm1l = [np.round(grouped_prompt1["mse"][2],2),np.round(grouped_prompt2["mse"][2],2),np.round(grouped_norm["mse"][2],2)]
ax[0, 2].bar(ettm1n, ettm1,color  =bar_colors ,edgecolor="k",label=ettm1l)
ax[0, 2].set_ylabel('MSE Values')
ax[0, 2].set_xlabel('Data Processing')
ax[0, 2].grid(False)
ax[0, 2].set_title('MSE from ETTm1')
ax[0, 2].legend(title='MSE Values')

ax[0, 2].spines['top'].set_visible(True)
ax[0, 2].spines['right'].set_visible(True)
ax[0, 2].spines['bottom'].set_visible(True)
ax[0, 2].spines['left'].set_visible(True)


#ETTm2

ettm2= [grouped_prompt1["mse"][3],grouped_prompt2["mse"][3],grouped_norm["mse"][3]]
ettm2n= ["prompt1","prompt2","prompt3"]
ettm2l = [np.round(grouped_prompt1["mse"][3],2),np.round(grouped_prompt2["mse"][3],2),np.round(grouped_norm["mse"][3],2)]
ax[1, 0].bar(ettm2n, ettm2,color  =bar_colors,edgecolor="k",label=ettm2l)
ax[1, 0].set_ylabel('MSE Values')
ax[1, 0].set_xlabel('Data Processing')
ax[1, 0].grid(False)
ax[1, 0].set_title('MSE from ETTm2')
ax[1, 0].legend(title='MSE Values')


ax[1, 0].spines['top'].set_visible(True)
ax[1, 0].spines['right'].set_visible(True)
ax[1, 0].spines['bottom'].set_visible(True)
ax[1, 0].spines['left'].set_visible(True)


#Heart

Heart= [grouped_prompt1["mse"][4],grouped_prompt2["mse"][4],grouped_norm["mse"][4]]
Heartn= ["prompt1","prompt2","prompt3"]
hl= [np.round(grouped_prompt1["mse"][4],2),np.round(grouped_prompt2["mse"][4],2),np.round(grouped_norm["mse"][4],2)]
ax[1, 1].bar(Heartn, Heart,color  =bar_colors,edgecolor="k",label=hl )
ax[1, 1].grid(False)
ax[1, 1].set_ylabel('MSE Values')
ax[1, 1].set_xlabel('Data Processing')

ax[1, 1].set_title('MSE from Heart')
ax[1, 1].legend(title='MSE Values')

ax[1, 1].spines['top'].set_visible(True)
ax[1, 1].spines['right'].set_visible(True)
ax[1, 1].spines['bottom'].set_visible(True)
ax[1, 1].spines['left'].set_visible(True)


#Suns
Suns= [grouped_prompt1["mse"][5],grouped_prompt2["mse"][5],grouped_norm["mse"][5]]
Sunsn= ["prompt1","prompt2","prompt3"]
sl = [np.round(grouped_prompt1["mse"][5],2),np.round(grouped_prompt2["mse"][5],2),np.round(grouped_norm["mse"][5],2)]
ax[1, 2].bar(Sunsn, Suns,color  =bar_colors,edgecolor="k",label = sl )
ax[1, 2].set_ylabel('MSE Values')
ax[1, 2].set_xlabel('Data Processing')
ax[1, 2].grid(False)
ax[1, 2].set_title('MSE from Sun')
ax[1, 2].legend(title='MSE Values')


ax[1, 2].spines['top'].set_visible(True)
ax[1, 2].spines['right'].set_visible(True)
ax[1, 2].spines['bottom'].set_visible(True)
ax[1, 2].spines['left'].set_visible(True)


#plt.show()
fig.savefig('BarPltMSE_prompts.png')
plt.close() 

In [67]:
bar_colors = ['tab:red', 'tab:blue', 'tab:orange','tab:green']
#bar_labels = [str(grouped["mse"][0]), str(grouped["mse"][1]), str(grouped["mse"][2]), str(grouped["mse"][3]),str(grouped["mse"][4]),str(grouped["mse"][5])]

fig, ax = plt.subplots(2,3) 
fig.set_figheight(15)
fig.set_figwidth(15)



#ETTh1
etth1= [grouped_prompt1["mae"][0],grouped_prompt2["mae"][0],grouped_norm["mae"][0]]
etth1l = [np.round(grouped_prompt1["mae"][0],2),np.round(grouped_prompt2["mae"][0],2),np.round(grouped_norm["mae"][0],2)]
etth1n= ["prompt1","prompt2","prompt3"]
ax[0, 0].bar(etth1n, etth1,color  =bar_colors, edgecolor="k" ,label= etth1l)
ax[0, 0].grid(False)
ax[0, 0].set_ylabel('MAE Values')
ax[0, 0].set_xlabel('Data Processing')

ax[0, 0].set_title('MAE from ETTh1')
ax[0, 0].legend(title='MAE Values')

ax[0, 0].spines['top'].set_visible(True)
ax[0, 0].spines['right'].set_visible(True)
ax[0, 0].spines['bottom'].set_visible(True)
ax[0, 0].spines['left'].set_visible(True)




#ETTh2
etth2= [grouped_prompt1["mae"][1],grouped_prompt2["mae"][1],grouped_norm["mae"][1]]
etth2l = [np.round(grouped_prompt1["mae"][1],2),np.round(grouped_prompt2["mae"][1],2),np.round(grouped_norm["mae"][1],2)]
etth2n= ["prompt1","prompt2","prompt3"]
ax[0, 1].bar(etth2n, etth2,color  =bar_colors,edgecolor="k",label= etth2l)
ax[0, 1].grid(False)
ax[0, 1].set_ylabel('MAE Values')
ax[0, 1].set_xlabel('Data Processing')

ax[0, 1].set_title('MAE from ETTh2')
ax[0, 1].legend(title='MAE Values')

ax[0, 1].spines['top'].set_visible(True)
ax[0, 1].spines['right'].set_visible(True)
ax[0, 1].spines['bottom'].set_visible(True)
ax[0, 1].spines['left'].set_visible(True)



#ETTm1
ettm1= [grouped_prompt1["mae"][2],grouped_prompt2["mae"][2],grouped_norm["mae"][2]]
ettm1n= ["prompt1","prompt2","prompt3"]
ettm1l = [np.round(grouped_prompt1["mae"][2],2),np.round(grouped_prompt2["mae"][2],2),np.round(grouped_norm["mae"][2],2)]
ax[0, 2].bar(ettm1n, ettm1,color  =bar_colors ,edgecolor="k",label=ettm1l)
ax[0, 2].set_ylabel('MAE Values')
ax[0, 2].set_xlabel('Data Processing')
ax[0, 2].grid(False)
ax[0, 2].set_title('MAE from ETTm1')
ax[0, 2].legend(title='MAE Values')

ax[0, 2].spines['top'].set_visible(True)
ax[0, 2].spines['right'].set_visible(True)
ax[0, 2].spines['bottom'].set_visible(True)
ax[0, 2].spines['left'].set_visible(True)


#ETTm2

ettm2= [grouped_prompt1["mae"][3],grouped_prompt2["mae"][3],grouped_norm["mae"][3]]
ettm2n= ["prompt1","prompt2","prompt3"]
ettm2l = [np.round(grouped_prompt1["mae"][3],2),np.round(grouped_prompt2["mae"][3],2),np.round(grouped_norm["mae"][3],2)]
ax[1, 0].bar(ettm2n, ettm2,color  =bar_colors,edgecolor="k",label=ettm2l)
ax[1, 0].set_ylabel('MAE Values')
ax[1, 0].set_xlabel('Data Processing')
ax[1, 0].grid(False)
ax[1, 0].set_title('MAE from ETTm2')
ax[1, 0].legend(title='MAE Values')

ax[1, 0].spines['top'].set_visible(True)
ax[1, 0].spines['right'].set_visible(True)
ax[1, 0].spines['bottom'].set_visible(True)
ax[1, 0].spines['left'].set_visible(True)


#Heart

Heart= [grouped_prompt1["mae"][4],grouped_prompt2["mae"][4],grouped_norm["mae"][4]]
Heartn= ["prompt1","prompt2","prompt3"]
hl= [np.round(grouped_prompt1["mae"][4],2),np.round(grouped_prompt2["mae"][4],2),np.round(grouped_norm["mae"][4],2)]
ax[1, 1].bar(Heartn, Heart,color  =bar_colors,edgecolor="k",label=hl )
ax[1, 1].grid(False)
ax[1, 1].set_ylabel('MAE Values')
ax[1, 1].set_xlabel('Data Processing')

ax[1, 1].set_title('MAE from Heart')
ax[1, 1].legend(title='MAE Values')

ax[1, 1].spines['top'].set_visible(True)
ax[1, 1].spines['right'].set_visible(True)
ax[1, 1].spines['bottom'].set_visible(True)
ax[1, 1].spines['left'].set_visible(True)

#Suns
Suns= [grouped_prompt1["mae"][5],grouped_prompt2["mae"][5],grouped_norm["mae"][5]]
Sunsn= ["prompt1","prompt2","prompt3"]
sl = [np.round(grouped_prompt1["mae"][5],2),np.round(grouped_prompt2["mae"][5],2),np.round(grouped_norm["mae"][5],2)]
ax[1, 2].bar(Sunsn, Suns,color  =bar_colors,edgecolor="k",label = sl )
ax[1, 2].set_ylabel('MAE Values')
ax[1, 2].set_xlabel('Data Processing')
ax[1, 2].grid(False)
ax[1, 2].set_title('MAE from Sun')
ax[1, 2].legend(title='MAE Values')


ax[1, 2].spines['top'].set_visible(True)
ax[1, 2].spines['right'].set_visible(True)
ax[1, 2].spines['bottom'].set_visible(True)
ax[1, 2].spines['left'].set_visible(True)

#plt.show()
fig.savefig('BarPltMAE_prompts.png')
plt.close() 

In [ ]:
MAE

In [66]:
bar_colors = ['tab:red', 'tab:blue', 'tab:orange','tab:green']
#bar_labels = [str(grouped["mse"][0]), str(grouped["mse"][1]), str(grouped["mse"][2]), str(grouped["mse"][3]),str(grouped["mse"][4]),str(grouped["mse"][5])]

fig, ax = plt.subplots(2,3) 
fig.set_figheight(15)
fig.set_figwidth(15)



#ETTh1
etth1= [grouped_prompt1["mape"][0],grouped_prompt2["mape"][0],grouped_norm["mape"][0]]
etth1l = [np.round(grouped_prompt1["mape"][0],2),np.round(grouped_prompt2["mape"][0],2),np.round(grouped_norm["mape"][0],2)]
etth1n= ["prompt1","prompt2","prompt3"]
ax[0, 0].bar(etth1n, etth1,color  =bar_colors, edgecolor="k" ,label= etth1l)
ax[0, 0].grid(False)
ax[0, 0].set_ylabel('MAPE Values')
ax[0, 0].set_xlabel('Data Processing')

ax[0, 0].set_title('MAPE from ETTh1')
ax[0, 0].legend(title='MAPE Values')

ax[0, 0].spines['top'].set_visible(True)
ax[0, 0].spines['right'].set_visible(True)
ax[0, 0].spines['bottom'].set_visible(True)
ax[0, 0].spines['left'].set_visible(True)




#ETTh2
etth2= [grouped_prompt1["mape"][1],grouped_prompt2["mape"][1],grouped_norm["mape"][1]]
etth2l = [np.round(grouped_prompt1["mape"][1],2),np.round(grouped_prompt2["mape"][1],2),np.round(grouped_norm["mape"][1],2)]
etth2n= ["prompt1","prompt2","prompt3"]
ax[0, 1].bar(etth2n, etth2,color  =bar_colors,edgecolor="k",label= etth2l)
ax[0, 1].grid(False)
ax[0, 1].set_ylabel('MAPE Values')
ax[0, 1].set_xlabel('Data Processing')

ax[0, 1].set_title('MAPE from ETTh2')
ax[0, 1].legend(title='MAPE Values')

ax[0, 1].spines['top'].set_visible(True)
ax[0, 1].spines['right'].set_visible(True)
ax[0, 1].spines['bottom'].set_visible(True)
ax[0, 1].spines['left'].set_visible(True)



#ETTm1
ettm1= [grouped_prompt1["mape"][2],grouped_prompt2["mape"][2],grouped_norm["mape"][2]]
ettm1n= ["prompt1","prompt2","prompt3"]
ettm1l = [np.round(grouped_prompt1["mape"][2],2),np.round(grouped_prompt2["mape"][2],2),np.round(grouped_norm["mape"][2],2)]
ax[0, 2].bar(ettm1n, ettm1,color  =bar_colors ,edgecolor="k",label=ettm1l)
ax[0, 2].set_ylabel('MAPE Values')
ax[0, 2].set_xlabel('Data Processing')
ax[0, 2].grid(False)
ax[0, 2].set_title('MAPE from ETTm1')
ax[0, 2].legend(title='MAPE Values')

ax[0, 2].spines['top'].set_visible(True)
ax[0, 2].spines['right'].set_visible(True)
ax[0, 2].spines['bottom'].set_visible(True)
ax[0, 2].spines['left'].set_visible(True)


#ETTm2

ettm2= [grouped_prompt1["mape"][3],grouped_prompt2["mape"][3],grouped_norm["mape"][3]]
ettm2n= ["prompt1","prompt2","prompt3"]
ettm2l = [np.round(grouped_prompt1["mape"][3],2),np.round(grouped_prompt2["mape"][3],2),np.round(grouped_norm["mape"][3],2)]
ax[1, 0].bar(ettm2n, ettm2,color  =bar_colors,edgecolor="k",label=ettm2l)
ax[1, 0].set_ylabel('MAPE Values')
ax[1, 0].set_xlabel('Data Processing')
ax[1, 0].grid(False)
ax[1, 0].set_title('MAPE from ETTm2')
ax[1, 0].legend(title='MAPE Values')

ax[1, 0].spines['top'].set_visible(True)
ax[1, 0].spines['right'].set_visible(True)
ax[1, 0].spines['bottom'].set_visible(True)
ax[1, 0].spines['left'].set_visible(True)


#Heart

Heart= [grouped_prompt1["mape"][4],grouped_prompt2["mape"][4],grouped_norm["mape"][4]]
Heartn= ["prompt1","prompt2","prompt3"]
hl= [np.round(grouped_prompt1["mape"][4],2),np.round(grouped_prompt2["mape"][4],2),np.round(grouped_norm["mape"][4],2)]
ax[1, 1].bar(Heartn, Heart,color  =bar_colors,edgecolor="k",label=hl )
ax[1, 1].grid(False)
ax[1, 1].set_ylabel('MAPE Values')
ax[1, 1].set_xlabel('Data Processing')

ax[1, 1].set_title('MAPE from Heart')
ax[1, 1].legend(title='MAPE Values')

ax[1, 1].spines['top'].set_visible(True)
ax[1, 1].spines['right'].set_visible(True)
ax[1, 1].spines['bottom'].set_visible(True)
ax[1, 1].spines['left'].set_visible(True)

#Suns
Suns= [grouped_prompt1["mape"][5],grouped_prompt2["mape"][5],grouped_norm["mape"][5]]
Sunsn= ["prompt1","prompt2","prompt3"]
sl = [np.round(grouped_prompt1["mape"][5],2),np.round(grouped_prompt2["mape"][5],2),np.round(grouped_norm["mape"][5],2)]
ax[1, 2].bar(Sunsn, Suns,color  =bar_colors,edgecolor="k",label = sl )
ax[1, 2].set_ylabel('MAPE Values')
ax[1, 2].set_xlabel('Data Processing')
ax[1, 2].grid(False)
ax[1, 2].set_title('MAPE from Sun')
ax[1, 2].legend(title='MAPE Values')


ax[1, 2].spines['top'].set_visible(True)
ax[1, 2].spines['right'].set_visible(True)
ax[1, 2].spines['bottom'].set_visible(True)
ax[1, 2].spines['left'].set_visible(True)

#plt.show()
fig.savefig('BarPltMAPE_prompts.png')
plt.close() 

In [102]:
def plot_time_series1(gt, series1,series2,series3,empty_array,input,path,name):
    """
    Plot two time series arrays.
    
    Args:
    series1 (list or numpy array): First time series data.
    series2 (list or numpy array): Second time series data.
    """
    gt = np.append(input,gt)
    series1 = np.append(empty_array,series1)
    series2 = np.append(empty_array,series2)
    series3 = np.append(empty_array,series3)
    
    # Generate time indices
    time_indices = list(range(1, len(series1) + 1))
   
    # Plot time series
    plt.figure(figsize=(10, 6))
    plt.plot(time_indices, gt, label='gt',color = "k")
    
    plt.plot(time_indices, series1, label='prompt1',color = "b")
    plt.plot(time_indices, series2, label='prompt2',color = "r")
    plt.plot(time_indices, series3, label='prompt3',color = "orange")
    
    # Add labels and legend
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.title(name)
    plt.legend()
    #plt.text(0.5, 0.5,str(mse), fontsize=14, transform=plt.gcf().transFigure)
    
    # Show plot
    plt.grid(True)

    path= path +"/"+ name + ".png"
    plt.savefig(path)
    plt.close() 
    #plt.show

In [82]:
def create_nan_array(length):
    return np.full(length, np.nan)
nan_array = create_nan_array(100)



# Load Grund Truth

In [83]:
path = "encoded/Encoded_final_test_dataset.json"

gt = load_dataset("json", data_files=path, split="train")


In [103]:
for i in range(len(prompt1)):

    inv_values_prompt1 = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],prompt1["answer"][i],mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
    #print(inv_values_prompt1)
    inv_values_prompt2 = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],prompt2["answer"][i],mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
    #print(inv_values_prompt2)
    inv_values_norm = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],norm_df["answer"][i],mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
    
    #print(inv_values_prompt1)
    plot_time_series1(deserialize_str(gt["output"][i],settings),inv_values_prompt1,inv_values_prompt2,inv_values_norm,nan_array,deserialize_str(gt["input"][i],settings)[-100:],"Exp2Plots",norm_df["dataset_name"][i])
    